# READ AND WRITE GOOGLESHEET : REPERE FONCTIONNEL

In [ ]:
from google.oauth2.service_account import Credentials
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
import re
import time
import sys

# Save timestamp
start = time.time()

# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = 'evn_param/gdriveteiaconnector-2ee95bd807fb.json' #bimgemdalkia@gmail.com

# Define the scope for the API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Authenticate and initialize gspread client
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(credentials)

# Google Sheet URL
sheet_url = "https://docs.google.com/spreadsheets/d/10-1NxN9YguPc7P0bPowJoCJ42BvO6_LKy0mav4lLmX4/edit"

# Other paramters
worksheet_name = 'Input_FNC'
output_worksheet_name = 'Demo Copie de ' + worksheet_name
fnc_des, all_matches, all_matches_string, match_bcf = [], [], [], []
new_column_header = 'Repère fonctionel'
target_fnc = '|'.join(['.tudes', 'Technique'])

# Regex pattern
pattern = r"\d[A-Z]{3}\d{3}[A-Z]{2}"

# Open the Google Sheet by URL
spreadsheet = client.open_by_url(sheet_url)
input_fnc = spreadsheet.worksheet(worksheet_name)



# Read data from the sheet
data = input_fnc.get_all_records()
#print("Data from sheet:", data[0]['FNC_Description'])
# Convert the data to a pandas DataFrame
df_fnc_brut = pd.DataFrame(data)
df_fnc = df_fnc_brut[df_fnc_brut["FNC_Type"].str.contains(target_fnc, case=False, na=False)]

# Exclude rows where the 'City' column is exactly underscore or empty
df_fnc = df_fnc[~df_fnc["FNC_Description"].isin(['_', '', '-'])]


#sys.exit()


#Get fnc description, option 1
# for i in (range(len(data))):
#     fnc_des.append(data[i]['FNC_Description'])
    
#print("FNC descriptions :",  fnc_des)

#Get fnc description, option 2
fnc_des = df_fnc["FNC_Description"]
#print(len(fnc_des))


print('Checking...')
for descript in fnc_des :
    # Remove spaces and underscores
    descript = descript.replace(" ", "").replace("_", "")
    
    # Get all the substring within a string that match a given pattern
    # Find all matches
    matches = re.findall(pattern, descript )
    # Convert the list to a set to remove duplicates, then convert it back to a list
    matches = list(set(matches))

    # Print the matches
    #print("Matches found:", matches)
    if len(matches)>0:
        all_matches.append(matches)
        all_matches_string.append(", ".join(matches))
        match_bcf.append(matches[0])
    else:
        all_matches.append('no match found')
        all_matches_string.append('no match found')
        match_bcf.append('no match found')



# Write data to the sheet : option 1 K.O because it exceed Google API limit
# Add the header to the first row (if not already present)
# header_row = input_fnc.row_values(1)

# if new_column_header not in header_row:
#     input_fnc.add_cols(1)  # Add a new column if it doesn't already exist
#     input_fnc.update_cell(1, len(header_row) + 1, new_column_header)  # Set the new header

# #Add values to the new column (starting from the second row)
# if new_column_header not in header_row:
#     for i, matches in enumerate(all_matches, start=2):
#         values_string = ", ".join(matches)
#         input_fnc.update_cell(i, len(header_row) + 1, values_string)
# else:
#     for i, matches in enumerate(all_matches, start=2):
#         values_string = ", ".join(matches)
#         input_fnc.update_cell(i, len(header_row), values_string)


# Write data to the sheet : option 2

df_fnc["Repere fonctionnel"] = all_matches_string
rows, cols = len(df_fnc), len(df_fnc.columns)

# Access or create the worksheet
try:
    output_worksheet = spreadsheet.worksheet(output_worksheet_name)
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{output_worksheet_name}' not found. Creating it.")
    output_worksheet = spreadsheet.add_worksheet(title=output_worksheet_name, rows=str(rows), cols=str(cols))


# Write DataFrame to the worksheet
set_with_dataframe(output_worksheet, df_fnc)
        
# Save timestamp
end = time.time()

print("DataFrame written to Google Sheets successfully!")
print("Elapsed time: ", end - start)

#Add the repere fonctionnel for maching with Revit Id

df_fnc["DSB_Repere fonctionnel"] = match_bcf
#print(df_fnc.head(7))



# ACC AUTHENTICATION 2LEGGEDTOKEN AND 3LEGGEDTOKEN 

In [ ]:
import requests
from Auth import Auth

# Replace these with your Forge App credentials
app_client_id = "DTU0A6uXmZ10LbJRalwDTAKlOIFmCi6N7rWWfVzSs14foADj"
app_client_secret = "9hu3hqUjTkJkA2GGEnHGA07GJvpCXNl3uJsmCCs6FOMoKLqMOA8pyvUGNgSvo727"

token_3leg = Auth(client_id = app_client_id, client_secret = app_client_secret).auth3leg(scopes = 'data:read data:write data:create data:search bucket:create bucket:read bucket:update bucket:delete code:all' ).access_token
print(token_3leg)



# ACC AUTHENTICATION - Get Hub ID

In [ ]:
import requests

# Replace these with your Forge App credentials
client_id = "DTU0A6uXmZ10LbJRalwDTAKlOIFmCi6N7rWWfVzSs14foADj"
client_secret = "9hu3hqUjTkJkA2GGEnHGA07GJvpCXNl3uJsmCCs6FOMoKLqMOA8pyvUGNgSvo727"

def get_access_token(): 
    url = "https://developer.api.autodesk.com/authentication/v2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "data:read"
    }
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    return response.json()["access_token"]

# Fetch the access token
try:
    access_token = get_access_token()
    print("Access Token:", access_token)
except requests.exceptions.HTTPError as err:
    print(f"HTTPError: {err}")
    
# Get Project ID
def get_hub_id(access_token):
    url = "https://developer.api.autodesk.com/project/v1/hubs"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    hubs = response.json().get("data", [])
    # for hub in hubs:
    #     print(f"Hub Name: {hub['attributes']['name']}")
    #     print(f"Hub ID: {hub['id']}")
    #     if hub["type"] == "hubs" and hub["attributes"]["extension"]["type"] == "projects":
    #         return hub["id"]  # Return the first hub of type 'projects'
    return None
hub_id = get_hub_id(access_token)    


# GET FILE URN ON ACC

In [ ]:
import requests

# Forge credentials
client_id = "DTU0A6uXmZ10LbJRalwDTAKlOIFmCi6N7rWWfVzSs14foADj"
client_secret = "9hu3hqUjTkJkA2GGEnHGA07GJvpCXNl3uJsmCCs6FOMoKLqMOA8pyvUGNgSvo727"

# Forge URLs
base_url = "https://developer.api.autodesk.com"

#Project name
project_name = "CCL-POC-FNC"
parent_folder_name = "Project Files" #default value
folder_name = "Test POC"
# Get OAuth Token

def get_access_token():
    url = "https://developer.api.autodesk.com/authentication/v2/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
        "scope": "data:read"
    }
    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    return response.json().get("access_token")

# Get Project ID
def get_hub_id(access_token):
    url = "https://developer.api.autodesk.com/project/v1/hubs"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    hubs = response.json().get("data", [])
    for hub in hubs:
        print(f"Hub Name: {hub['attributes']['name']}")
        print(f"Hub ID: {hub['id']}")
        #if hub["type"] == "hubs" and hub["attributes"]["extension"]["type"] == "projects":
            #return hub["id"]  # Return the first hub of type 'projects'
    return hub["id"] if hubs else None 

def get_projects(hub_id, access_token):
    url = f"{base_url}/project/v1/hubs/{hub_id}/projects"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    projects = response.json().get("data", [])
    return projects if projects else None #return all projects
    #return projects[0]["id"] if projects else None  # Example: returns the first project

def get_proj_id_fr_proj_name(projects, project_name):
    if projects :
        for project in projects :
            if project["attributes"]["name"] == project_name:
                project_id = project["id"]
        return project["id"] 
    else :
        return None
    
# Get folder id


# url = f"https://developer.api.autodesk.com/project/v1/projects/{project_id}/folders"
# headers = {
#     "Authorization": f"Bearer {access_token}"
# }

# # Get the folders in the project
# response = requests.get(url, headers=headers)
# response.raise_for_status()

# # Parse the response to get folder IDs
# folders = response.json()["data"]
# for folder in folders:
#     folder_id = folder["id"]
#     folder_name = folder["attributes"]["name"]
#     print(f"Folder ID: {folder_id}, Folder Name: {folder_name}")

def get_topFolders(hub_id, project_id, access_token):
    #url = f"{base_url}/project/v1/hubs/{hub_id}/projects"
    url = f"https://developer.api.autodesk.com/project/v1/hubs/{hub_id}/projects/{project_id}/topFolders"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    topfolders = response.json().get("data", [])
    return  topfolders if topfolders else None #return all top folders

# Get Folder Contents
def get_folder_contents(project_id, folder_id, access_token):
    url = f"{base_url}/data/v1/projects/{project_id}/folders/{folder_id}/contents"
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    return response.json()

def get_folderId_fr_folderName(topfolders,parent_folder_name):
    if topfolders :
        for topfolder in topfolders :
            if topfolder["attributes"]["name"] == parent_folder_name :
                parent_folder_id = topfolder["id"]     
                print(f"Parent folder ID: {parent_folder_id }, Parent folder Name: {parent_folder_name}")
        return parent_folder_id 


# Main workflow
from aps_toolkit import Auth
import warnings
warnings.filterwarnings("ignore")


#ACC Authentification
access_token = get_access_token()

# Replace these values with actual ones
#hub_id = "your_hub_id"  # Obtain this from the ACC admin console or API
hub_id = get_hub_id(access_token)


# # Get project and folder contents
projects = get_projects(hub_id, access_token)
project_id = get_proj_id_fr_proj_name(projects, project_name)

print("project_id of ", project_name, " : ", project_id)

topfolders = get_topFolders(hub_id, project_id, access_token)

parent_folder_id = get_folderId_fr_folderName(topfolders,parent_folder_name)

parent_folder_contents = get_folder_contents(project_id, parent_folder_id, access_token)

# Get folder Id:

if project_id : 
    for folder in parent_folder_contents.get("data", []) : 
        
        if folder["type"]=='folders' and folder['attributes']['name'] == 'Test POC':
            folder_id = folder["id"]
            folder_contents = get_folder_contents(project_id, folder_id, access_token)
        
            for item in folder_contents.get("included", []):
                if ".rvt" in item['attributes']['name']:  # Check if it's a revit file
                    print(f"File: {item['attributes']['displayName']}")
                    print(f"URN: {item['relationships']['derivatives']['data']['id']}")
                
                
    

# ACC API ITO : REVIT ID, IfcGuid, Famille et type, DSB_Repere fonctionnel

In [ ]:
from revit_meows import APSRevit
from aps_toolkit import Auth
import warnings
warnings.filterwarnings("ignore")
import requests
import pandas as pd


from Auth import Auth

project_id = 'ee830b5c-fb1d-4fdb-a013-76928787b9c3' #to make it work here, the project id must not contain "b."
# Replace these with your Forge App credentials
app_client_id = "DTU0A6uXmZ10LbJRalwDTAKlOIFmCi6N7rWWfVzSs14foADj"
app_client_secret = "9hu3hqUjTkJkA2GGEnHGA07GJvpCXNl3uJsmCCs6FOMoKLqMOA8pyvUGNgSvo727"

# Get OAuth Token
token_3leg = Auth(client_id = app_client_id, client_secret = app_client_secret).auth3leg(scopes = 'data:read data:write data:create data:search bucket:create bucket:read bucket:update bucket:delete code:all' ).access_token
print(token_3leg)


urn  = "dXJuOmFkc2sud2lwZW1lYTpmcy5maWxlOnZmLklvQnZxbGkxU18yZzBqU01TdndreGc_dmVyc2lvbj0y"
version_urn = "urn:adsk.wipemea:fs.file:vf.IoBvqli1S_2g0jSMSvwkxg?version=2"

revit_data = APSRevit(urn,token_3leg, region="EMEA")
#revit_data_df = revit_data.get_all_data()
revit_data_df2 = revit_data.get_all_data_bbox(project_id)


revit_data_df2.head(10)




# EXPORT ACC DATA TO EXCEL

In [ ]:
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = 'evn_param/gdriveteiaconnector-2ee95bd807fb.json' #bimgemdalkia@gmail.com

# Define the scope for the API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Authenticate and initialize gspread client
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(credentials)

# Google Sheet URL
sheet_url = "https://docs.google.com/spreadsheets/d/10-1NxN9YguPc7P0bPowJoCJ42BvO6_LKy0mav4lLmX4/edit"

# Open the Google Sheet by URL
spreadsheet = client.open_by_url(sheet_url)
worksheet_name = "ACC_ITO"
worksheet_BCF = "BCF_data"

revit_expdata_df = revit_data_df2.filter(items=['ElementId', 'IfcGUID', 'objectid','Family Name','DSB_Repere fonctionnel', 'bboxMin.x', 'bboxMin.y', 'bboxMin.z', 'bboxMax.x', 'bboxMax.y', 'bboxMax.z']) #filter only the necessary columns 
#revit_expdata_df = revit_data_df2.head(10) #filter first 10 rows to get all paramters
rows, cols = len(revit_expdata_df), len(revit_expdata_df.columns)
#print(rows, cols)


# Access or create the worksheet
try:
    worksheet = spreadsheet.worksheet(worksheet_name)
except gspread.exceptions.WorksheetNotFound:
    print(f"Worksheet '{worksheet_name}' not found. Creating it.")
    worksheet = spreadsheet.add_worksheet(title=worksheet_name, rows=str(rows), cols=str(cols))


# Write DataFrame to the worksheet
set_with_dataframe(worksheet, revit_expdata_df)
print("ACC ITO written to Google Sheets successfully!")



#Maching by combining df_fnc with revit_expdata_df:
# try :
#     df_bcf = pd.merge(df_fnc, revit_expdata_df, how='inner', on='DSB_Repere fonctionnel')
#     print('Matching FNC Data with ACC Data successfully!')
# except:
#     print('Matching FNC Data with ACC Data failed!')

# # Access or create the worksheet
# rows_bcf, cols_bcf = len(df_bcf), len(df_bcf.columns)
# try:
#     bcf_worksheet = spreadsheet.worksheet(worksheet_BCF)
# except gspread.exceptions.WorksheetNotFound:
#     print(f"Worksheet '{worksheet_BCF}' not found. Creating it.")
#     bcf_worksheet = spreadsheet.add_worksheet(title=worksheet_BCF, rows=str(rows), cols=str(cols))
    
# # Write DataFrame to the worksheet
# set_with_dataframe(bcf_worksheet, df_bcf)
# print("BCF Data written to Google Sheets successfully!")

# ACC API POST : ISSUE CREATION

In [ ]:
# The objectif of this script is to create an issue following https://aps.autodesk.com/en/docs/acc/v1/tutorials/issues/create-issues/

import requests

#ACC Authentification
app_client_id = "DTU0A6uXmZ10LbJRalwDTAKlOIFmCi6N7rWWfVzSs14foADj"
app_client_secret = "9hu3hqUjTkJkA2GGEnHGA07GJvpCXNl3uJsmCCs6FOMoKLqMOA8pyvUGNgSvo727"
project_id = 'ee830b5c-fb1d-4fdb-a013-76928787b9c3' #to make it work here, the project id must not contain "b."
m2f = 3.28084 #covert meter to feet

# Replace these with your Forge App credentials


token_3leg = Auth(client_id = app_client_id, client_secret = app_client_secret).auth3leg(scopes = 'data:read data:write data:create data:search bucket:create bucket:read bucket:update bucket:delete code:all' ).access_token

# Step 1: Verify the User’s Permissions
def verify_permission(project_id):
    url = f"https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/users/me"
    headers = {"Authorization": f"Bearer {token_3leg}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print("Response status:", response.status_code)
    print("Response body:", response.text)
    return response.json()


# Step 2: Find the Issue Type (subtype) ID
def find_issues_type(project_id):
    url = f"https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/issue-types?include=subtypes"
    headers = {"Authorization": f"Bearer {token_3leg}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    return response.json()

#Step 5: get location
def get_location_id(project_id):
    url = f"https://developer.api.autodesk.com/construction/locations/v2/projects/{project_id}/trees/default/nodes?limit=3&offset=3"
    headers = {"Authorization": f"Bearer {token_3leg}"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    #print("Response status:", response.status_code)
    #print("Response body:", response.text)
    return response.json()
#Step 6: Create the Issue

def create_issue_v1(project_id, linked_doc):
    url = f'https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/issues'
    headers = {"Authorization": f"Bearer {token_3leg}",
                "Content-Type": "application/json"}
    data = {
        "title": "Test API ACC Thi with element id and linked doc (objectId = 3257, position, draft, template_V1, meter to feet)",
        "description": "Test API ACC, ne pas prendre en compte, elementid = 7600210, DSB_Repere fonctionnel = 0SEJ801PO ",
        "issueSubtypeId": "cc9b2b0b-177f-48d8-894b-21e335af6654",
        "status": "draft",
        "linkedDocuments": linked_doc
    }
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print("Response status:", response.status_code)
    print("Response body:", response.text)
    return response.json()

def create_issue_v2(project_id, titre, description, issueSubtypeId, status, linked_doc):
    url = f'https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/issues'
    headers = {"Authorization": f"Bearer {token_3leg}",
                "Content-Type": "application/json"}
    data = {
        "title": titre,
        "description": description,
        #"issueSubtypeId": "cc9b2b0b-177f-48d8-894b-21e335af6654",
        "issueSubtypeId": issueSubtypeId,
        "status": status,
        "linkedDocuments": linked_doc
    }
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print("Response status:", response.status_code)
    print("Response body:", response.text)
    return response.json()

def get_issue(project_id):
    url = f'https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/issues'
    headers = {"Authorization": f"Bearer {token_3leg}",
                "Content-Type": "application/json"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print("Response status:", response.status_code)
    print("Response body:", response.text)
    return response.json()

#verify_permission(project_id)
#find_issues_type(project_id)
#get_location_id(project_id)



#Readjson file
import json

# Open and read the JSON file
with open('input/ACC_issue_template.json', 'r') as file:
    issue_template = json.load(file)

# Access the parsed data
linkedDocuments =  issue_template['linkedDocuments']
linkedDocuments[0]["details"]["objectId"] = 3257
linkedDocuments[0]["details"]['position']['x'] = -5.67018609940507*m2f
linkedDocuments[0]["details"]['position']['y'] = -23.7266215209961*m2f
linkedDocuments[0]["details"]['position']['z'] = -0.0000000434217506395385*m2f
# linkedDocuments[0]["details"]['viewerState']['viewport']['eye'] = [x,y,z]
# linkedDocuments[0]["details"]['viewerState']['viewport']["target"] = [x,y,z]
# linkedDocuments[0]["details"]['viewerState']['viewport']["pivotPoint"] = ["0","0","0"]

#print(linkedDocuments)
#get_issue(project_id)
#create_issue_v1(project_id, linkedDocuments)


# TEST : ISSUE CREATION FROM FNC DATA

In [ ]:
from google.oauth2.service_account import Credentials
import gspread
from gspread_dataframe import set_with_dataframe
import pandas as pd
import re
import time
import sys



# Path to your service account JSON key file
SERVICE_ACCOUNT_FILE = 'evn_param/gdriveteiaconnector-2ee95bd807fb.json' #bimgemdalkia@gmail.com

# Define the scope for the API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Authenticate and initialize gspread client
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
client = gspread.authorize(credentials)

# Google Sheet URL
sheet_url = "https://docs.google.com/spreadsheets/d/10-1NxN9YguPc7P0bPowJoCJ42BvO6_LKy0mav4lLmX4/edit"

# Other paramters
worksheet_name = 'Demo_BCF_data'

# Open the Google Sheet by URL
spreadsheet = client.open_by_url(sheet_url)
bcf_data_gs = spreadsheet.worksheet(worksheet_name)

# Read data from the sheet
bcf_data = bcf_data_gs.get_all_records()
df_bcf_data = pd.DataFrame(bcf_data)
issueSubtypeId = "cc9b2b0b-177f-48d8-894b-21e335af6654"


#Readjson file
import json

# Open and read the JSON file
with open('input/ACC_issue_template.json', 'r') as file:
    issue_template = json.load(file)

def create_issue_v2(project_id, titre, description, issueSubtypeId, status, linked_doc, objectId):
    url = f'https://developer.api.autodesk.com/construction/issues/v1/projects/{project_id}/issues'
    headers = {"Authorization": f"Bearer {token_3leg}",
                "Content-Type": "application/json"}
    data = {
        "title": titre,
        "description": description,
        #"issueSubtypeId": "cc9b2b0b-177f-48d8-894b-21e335af6654",
        "issueSubtypeId": issueSubtypeId,
        "status": status,
        "linkedDocuments": linked_doc
    }
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()  # Raises an error for 4xx/5xx responses
    print("Response status:", response.status_code)
    #print("Response body:", response.text)
    return response.json()

linkedDocuments =  issue_template['linkedDocuments']
status = 'in_review'

for index, revit_id in enumerate(df_bcf_data["ElementId"]) :
    x =  df_bcf_data["Center.x"][index]
    y =  df_bcf_data["Center.y"][index]
    z =  df_bcf_data["Center.z"][index]
    titre = df_bcf_data["FNC_Titre"][index]
    description = df_bcf_data["FNC_Description"][index]
    objectId = df_bcf_data["objectId"][index]
    linkedDocuments[0]["details"]["objectId"] =  int(objectId)
    linkedDocuments[0]["details"]['position']['x'] = x*m2f
    linkedDocuments[0]["details"]['position']['y'] = y*m2f
    linkedDocuments[0]["details"]['position']['z'] = z*m2f
    create_issue_v2(project_id, titre, description, issueSubtypeId, status, linkedDocuments, objectId)
    
    